In [1]:
import numpy as np

from src import utils

In [2]:
corp = utils.load_corpus_from_files(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02')

# Check manual downbeat accuracy

In [3]:
man_res = []
auto_res = []
comp_res = []

for track in corp:
    if track.item['has_annotations']:
        # Get downbeats implied from Tessa's annotations
        db1 = track.item['first_downbeat']
        tracked = track.ons['mix']
        manual_db = tracked[np.argmin(abs(db1 - tracked))::track.item['time_signature']]
        # Get the automatically tracked downbeats
        auto_db = track.ons['downbeats_auto']
        # Get the ground truth downbeats
        fn = rf'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_mix.txt'
        txt = np.loadtxt(fn, delimiter='\t', usecols=[0, 1])
        # Subset ground truth beats to get only those marked as downbeats
        gt = np.array([ts for ts, met in txt if int(str(met).split('.')[-1]) == 1])
        # Compare accuracy
        auto_f = track.compare_onset_detection_accuracy(onsets=auto_db, ref=gt)['f_score']
        man_f = track.compare_onset_detection_accuracy(onsets=manual_db, ref=gt)['f_score']
        comp_f = track.compare_onset_detection_accuracy(onsets=manual_db, ref=auto_db)['f_score']
        # Append results to list
        man_res.append(man_f)
        auto_res.append(auto_f)
        comp_res.append(comp_f)

In [7]:
print(np.mean(man_res))

0.7071032590668818


In [8]:
print(np.mean(auto_res))

0.6279583903902597


In [9]:
print(np.mean(comp_res))

0.5645856146806403


# Check for octave errors in manual vs automatic annotations

In [10]:
def get_bar_avg_tempo(bar):
    return np.mean(60 / np.diff(bar))

def split_to_bars(beats, time_sig):
    # Iterate through each beat
    for ll in range(0, len(beats)):
        # Get the upper limit (equivalent to one bar and subset
        ul = ll + time_sig
        bar = beats[ll: ul]
        # If we've ran out of beats to process
        if len(bar) < time_sig:
            break
        yield bar
            

In [21]:
res = []
for track in corp:
    # Skip the track if we don't have ground truth annotations
    if not track.item['has_annotations']:
        continue
    # We can just use the beat tracking results here
    # Using the band timestamps won't change much
    auto_beats = track.summary_dict['beats']
    # Load in the manual beat tracking results
    fn = rf'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_mix.txt'
    man_beats = np.loadtxt(fn, delimiter='\t', usecols=[0])
    # Subset beats into bars
    auto_bars = split_to_bars(auto_beats, track.item['time_signature'] * 4)
    man_bars = split_to_bars(man_beats, track.item['time_signature'] * 4)
    # Get average tempo from bars
    auto_temps = np.array([get_bar_avg_tempo(bar) for bar in auto_bars])
    man_temps = np.array([get_bar_avg_tempo(bar) for bar in man_bars])
    # Get absolute differnce in mean tempo from automatic/manual annotations
    diff = auto_temps.mean() / man_temps.mean()
    res.append(diff)


In [23]:
print(np.median(res))
print(np.std(res))

0.9994909134614671
0.0027648771533463298


In [24]:
print(res)

[0.9997497407005782, 0.9897424536223364, 0.9997483482193757, 0.9995086395236125, 1.0039960031901765, 0.9977069811947354, 0.9998522660206451, 0.9984412963764357, 1.0011414353480854, 0.9986139608456287, 1.0074785768279984, 0.9977585363496678, 0.9985529132379081, 0.9993407453721729, 1.0026313356270806, 0.9995245573888077, 0.9966976449668349, 0.9960682137871205, 0.9989473269715174, 1.0000000000418905, 0.9992620311729388, 0.9994621358338489, 0.9997771846130616, 0.9997539783462085, 0.9995225972061664, 1.0010060421669582, 0.9991846149344242, 0.9995072650391544, 0.9944510096252323, 0.9985914291941757, 1.0026897216691721, 0.999100638547221, 1.0002046421103856, 0.9994745618837797]
